In [1]:
param_date = '20240715'

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [3]:
input_ = pd.read_excel(f'./{param_date}/1.manufacturer.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Manufacturer No'] = input_['Manufacturer No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Manufacturer'].tolist()

总数量：551


['3D',
 '3M',
 'AAE',
 'ACC',
 'ACCEL',
 'ACDELCO',
 'ACEON',
 'ACI',
 'ACME AUTO',
 'ADVAN-TECH',
 'ADVICS',
 'AEM INDUCTION',
 'AGILITY',
 'AGS',
 'AIMCO',
 'AIRAID',
 'AIRQUALITEE',
 'AIRTEX',
 'AISIN',
 'AJUSA',
 'AKEBONO',
 "AL'S LINER",
 'AL-KO',
 'ALBANY',
 'ALLIGATOR',
 'ALLPARTS',
 'ALO',
 'ALTROM',
 'AMERICAN COMPONENTS',
 'AMERICAN REMANUFACTURERS INC.',
 'AMS AUTOMOTIVE',
 'ANCHOR',
 'ANCO',
 'ANSA',
 'AP / EASTERN',
 'APEX',
 'API',
 'APWI',
 'ARIES',
 'ARMORMARK',
 'ARNOTT',
 'ARP',
 'ARROW AUTOMOTIVE',
 'ATE',
 'ATP',
 'ATSCO',
 'AUS INJECTION INC.',
 'AUTO 7',
 'AUTO EXTRA',
 'AUTO EXTRA/CHASSIS RITE',
 'AUTO PRIDE',
 'AUTO-TUNE',
 'AUTOBEST',
 'AUTODRIVE',
 'AUTOLINE',
 'AUTOLITE',
 'AUTOMETER',
 'AUTOMOTIVE UNDERCAR PRODUCTS',
 'AUTOSPECIALTY',
 'AUTOSPECIALTY/KELSEY-HAYES',
 'AUTOTECH ENGINEERING',
 'AUTOTECNICA',
 'AUTOTEMP',
 'AUVECO',
 'AVS',
 'B&B',
 "B'LASTER",
 'BALKAMP',
 'BANDO',
 'BB BRAKES',
 'BBB INDUSTRIES',
 'BCA',
 'BECK/ARNLEY',
 'BELDEN',
 'BENDIX',
 

In [4]:
a = 0

a

0

In [5]:
crawler_status = 'error'

crawler_status

'error'

In [6]:
import requests
import json

import sys
sys.path.append('../../00.Tools')
from crawler_configuration import get_header, get_proxy

data = {'func': 'partsearchgetpartgroupsdd',
        'mfr': input_.loc[a, 'Manufacturer'],
        'giveback': json.dumps({'id': 'partsearch_007'}),
        'api_json_request': '1'}

b = 0
while True:

    while True:
        b += 1
        print(b)
        try:
            resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                 data=data,
                                 headers=get_header(),
                                 proxies=get_proxy(),
                                 timeout=(10, 10))
            break
        except:
            continue

    if resp.status_code == 200:
        break

resp

1


<Response [200]>

In [7]:
json_ = resp.json()

json_

{'html_replace_sections': {'ddrepl[partgroup_][container]': '<span id="ddrepl[partgroup_][container]" class="ra-vertical-dropdown-replacement-c"><select id="partgroup_" name="partsearch[partgroup][]" onchange = "clb.ChangePartGroupOption(this, null);;" tabindex="0" autocomplete = "none"><option value=""  selected >All</option><option value="Body &amp; Lamp Assembly"  >Body &amp; Lamp Assembly</option><option value="Interior"  >Interior</option></select></span>'},
 'giveback': {'id': 'partsearch_007'},
 'collected_javascript': '/**UQ:A316B883**/\nwindow.top.parent.window["_nck"] = "8LVLUxTaILBT+tLxkg8+3Ag7\\/0bbOhvn7P3rkFbL809YNOHQDmNBznFtdYfbcEUriQ0mcmy\\/TcWfcbfHTjxSwndfe2chgzy88liHO2MoAcp7wW7lek5DxawTLSg\\/DpYUwwgKwrMfy32BI6R68c\\/w7jnBJG9yxQJUiSXsTLJjK84=";\n'}

In [ ]:
'redirect_to_url' in json_

In [8]:
json_['giveback']['id'] == 'partsearch_007' and 'collected_javascript' in json_

True

In [9]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(json_['html_replace_sections']['ddrepl[partgroup_][container]'], 'lxml')
html = etree.HTML(str(soup))

with open('./group.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <span class="ra-vertical-dropdown-replacement-c" id="ddrepl[partgroup_][container]">
   <select autocomplete="none" id="partgroup_" name="partsearch[partgroup][]" onchange="clb.ChangePartGroupOption(this, null);;" tabindex="0">
    <option selected="" value="">
     All
    </option>
    <option value="Body &amp; Lamp Assembly">
     Body &amp; Lamp Assembly
    </option>
    <option value="Interior">
     Interior
    </option>
   </select>
  </span>
 </body>
</html>



In [10]:
list_group = html.xpath('//select[@id="partgroup_"]/option/@value')[1:]

list_group

['Body & Lamp Assembly', 'Interior']

In [11]:
len(list_group) == 0

False

In [12]:
df_temp = pd.DataFrame({'Manufacturer No': input_.loc[a, 'Manufacturer No'],
                        'Manufacturer': input_.loc[a, 'Manufacturer'],
                        'Group No': [i+1 for i in range(len(list_group))],
                        'Group': list_group})

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,Manufacturer No,Manufacturer,Group No,Group
0,1,3D,1,Body & Lamp Assembly
1,1,3D,2,Interior


In [13]:
crawler_status = 'ok'

crawler_status

'ok'

In [14]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Manufacturer No': input_.loc[a, 'Manufacturer No'],
                             'Manufacturer': input_.loc[a, 'Manufacturer']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [15]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Manufacturer No', 'Group No'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_group.xlsx', index=False)

output_correct

,Manufacturer No,Manufacturer,Group No,Group
0,1,3D,1,Body & Lamp Assembly
1,1,3D,2,Interior


In [16]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['Manufacturer No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_group_error.xlsx', index=False)

output_error

""
